# 🤗 `transformers`

More info about using `transformers` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/transformers/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/examples/extraction/

In [ ]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.transformers import TransformersJSONGenerator

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [ ]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [ ]:
prompt_template = """You are the owner of a pizza parlor. Customers
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [ ]:
generator = TransformersJSONGenerator(model_name="microsoft/Phi-3-mini-4k-instruct", schema_object=Order, device=device)

### Standalone

In [ ]:
generator.warm_up()

In [ ]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt, generation_kwargs={"temperature": 0.5, "do_sample": True})

### In a Pipeline

In [ ]:
pipeline = Pipeline()
pipeline.add_component(
    instance=PromptBuilder(template=prompt_template, required_variables=["order"]), name="prompt_builder"
)
pipeline.add_component(
    instance=TransformersJSONGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        schema_object=Order,
        device=device,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run(
    {
        "prompt_builder": {"order": "Is it possible to have 12 margheritas?"},
        "llm": {"generation_kwargs": {"temperature": 0.5, "do_sample": True}},
    }
)

## Choice Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.transformers import TransformersChoiceGenerator

In [ ]:
generator = TransformersChoiceGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    choices=["Positive", "Negative"],
    device=device,
)

### Standalone

In [ ]:
generator.warm_up()

In [ ]:
generator.run(
    prompt="Classify the following statement: 'I love pizza'", generation_kwargs={"temperature": 0.5, "do_sample": True}
)

### In a Pipeline

In [ ]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(
    instance=PromptBuilder(template=prompt_template, required_variables=["statement"]), name="prompt_builder"
)
pipeline.add_component(
    instance=TransformersChoiceGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        choices=["Positive", "Negative"],
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run(
    {
        "prompt_builder": {"statement": "I love Italian food"},
        "llm": {"generation_kwargs": {"temperature": 0.5, "do_sample": True}},
    }
)

## Text Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.transformers import TransformersTextGenerator

In [ ]:
generator = TransformersTextGenerator(
    model_name="microsoft/Phi-3-mini-4k-instruct",
    device=device,
)

### Standalone

In [ ]:
generator.warm_up()

In [ ]:
generator.run(prompt="What is the capital of Italy?")

### In a Pipeline

In [ ]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(
    instance=PromptBuilder(template=prompt_template, required_variables=["country"]), name="prompt_builder"
)
pipeline.add_component(
    instance=TransformersTextGenerator(
        model_name="microsoft/Phi-3-mini-4k-instruct",
        device=device,
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run({"prompt_builder": {"country": "France"}})